In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input

In [2]:
# Load the ResNet50 model
input_tensor = Input(shape=(215, 215, 3))
resnet_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the layers in the base model so that they are not trained
for layer in resnet_model.layers:
    layer.trainable = False

In [3]:
# Define a new model that includes the base ResNet50 model as well as a new output layer
x = Flatten()(resnet_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)
model = tf.keras.models.Model(inputs=resnet_model.input, outputs=output)

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define data generators for the training and testing data
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory('C:/Users/Kranti/Desktop/EE541_project/archive/Training/', 
                                                    batch_size=200, 
                                                    target_size=(215,215), 
                                                    class_mode='categorical', 
                                                    classes=['notumor', 'tumor'])
test_generator = test_datagen.flow_from_directory('C:/Users/Kranti/Desktop/EE541_project/archive/Testing/', 
                                                  batch_size=200, 
                                                  target_size=(215,215), 
                                                  class_mode='categorical', 
                                                  classes=['notumor', 'tumor'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_generator)

# Print the test accuracy
print('Test accuracy:', accuracy)